In [2]:
import pandas as pd
import numpy as np
from sodapy import Socrata
# this step is to read public data, the only thing needs to know is how to read data from
# csv file: pd.read_csv('data.csv',sep = ',')
client = Socrata("health.data.ny.gov", None)
result = client.get("cr7a-34ka",limit = 1000)
ppv = pd.DataFrame.from_records(result)
ppv = ppv[['year','patient_county','dual_status','medicaid_ppv_events']]
# change column names
#ppv.columns = ['year','county','dual_status','medicaid_ppv_events']
ppv.rename(columns = {'patient_county':'county'},inplace = True)
ppv.head()

,year,county,dual_status,medicaid_ppv_events
0,2014,Albany,Non-Dual,28303
1,2014,Albany,Total,33297
2,2014,Albany,Dual,4994
3,2014,Allegany,Dual,864
4,2014,Allegany,Non-Dual,3852


In [9]:
type(ppv['county'].to_frame().head())

pandas.core.frame.DataFrame

In [12]:
ppv['county'].str.strip().to_frame().head()

,county
0,Albany
1,Albany
2,Albany
3,Allegany
4,Allegany


In [4]:
# remove space in county so we can join with tax data frame, just data cleaning
ppv['county'] = ppv['county'].str.strip()
# number of records
print(ppv.county.count())
# number of distinct records
print(ppv.county.nunique())

504
63


In [13]:
# see the output of size() function
ppv.groupby(['year','county']).size().head()

year  county      
2011  Albany          1
      Allegany        1
      Bronx           1
      Broome          1
      Cattaraugus     1
dtype: int64

In [6]:
# convert to data frame
ppv.groupby(['year','county']).size().reset_index().head()

,year,county,0
0,2011,Albany,1
1,2011,Allegany,1
2,2011,Bronx,1
3,2011,Broome,1
4,2011,Cattaraugus,1


In [40]:
ppv.groupby(['year'])['dual_status'].agg({'dual_status':'nunique', 'size2':'size', 'sum':'sum'})

/Users/exue001c/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,sum,dual_status,size2
year,,,
2011,Total Total Total Total Total Total Total Tota...,1,63
2012,Total Total Total Total Total Total Total Tota...,1,63
2013,Total DualNon DualDualTotal Non DualDualTotal ...,3,189
2014,Non-DualTotal DualDualNon-DualTotal Non-DualTo...,3,189


In [7]:
# see the output of count() function, how it differs from size() function
ppv.groupby(['year','county']).count().head()

dual_status  medicaid_ppv_events
year county                                       
2011 Albany                 1                    1
     Allegany               1                    1
     Bronx                  1                    1
     Broome                 1                    1
     Cattaraugus            1                    1

In [8]:
# convert to data frame
ppv.groupby(['year','county']).count().reset_index().head()

,year,county,dual_status,medicaid_ppv_events
0,2011,Albany,1,1
1,2011,Allegany,1,1
2,2011,Bronx,1,1
3,2011,Broome,1,1
4,2011,Cattaraugus,1,1


In [9]:
# another way to convert to data frame
ppv.groupby(['year','county']).size().to_frame('dual_status').reset_index().head()

,year,county,dual_status
0,2011,Albany,1
1,2011,Allegany,1
2,2011,Bronx,1
3,2011,Broome,1
4,2011,Cattaraugus,1


In [11]:
# unique number of dual status in each year
ppv.groupby('year')['dual_status'].nunique().reset_index()

,year,dual_status
0,2011,1
1,2012,1
2,2013,3
3,2014,3


In [14]:
# output of unique dual status
ppv.dual_status.unique()

array([u'Non-Dual', u'Total ', u'Dual', u'Non Dual'], dtype=object)

In [15]:
ppv1 = ppv.copy()
# change 'Non-Dual' to 'Non Dual' for dual_status column
ppv.loc[ppv.dual_status == 'Non-Dual','dual_status'] = 'Non Dual'

In [44]:
# change 'Non-Dual' to 'Non Dual' for dual_status column
ppv.loc[ppv.dual_status == 'Non-Dual','dual_status']='Non Dual'

In [45]:
ppv.dual_status.unique()

array(['Non Dual', u'Total ', u'Dual'], dtype=object)

In [17]:
cols = ['medicaid_ppv_events']
# change string to numeric variable, no need to know this 
ppv[cols] = ppv[cols].apply(pd.to_numeric, errors='coerce', axis=1)
# pivot table
ppv.pivot_table(index = ['year','county'], columns = 'dual_status', values = 'medicaid_ppv_events')

dual_status           Dual   Non Dual     Total 
year county                                     
2011 Albany            NaN        NaN    29612.0
     Allegany          NaN        NaN     3948.0
     Bronx             NaN        NaN   303162.0
     Broome            NaN        NaN     9121.0
     Cattaraugus       NaN        NaN     8012.0
     Cayuga            NaN        NaN     5385.0
     Chautauqua        NaN        NaN    16341.0
     Chemung           NaN        NaN    12741.0
     Chenango          NaN        NaN     6662.0
     Clinton           NaN        NaN     9798.0
     Columbia          NaN        NaN     5636.0
     Cortland          NaN        NaN     3796.0
     Delaware          NaN        NaN     4797.0
     Dutchess          NaN        NaN    14118.0
     Erie              NaN        NaN    62290.0
     Essex             NaN        NaN     4007.0
     Franklin          NaN        NaN     5723.0
     Fulton            NaN        NaN     7647.0
     Genesee           NaN        NaN     4216.0
     Greene            NaN        NaN     3322.0
     Hamilton          NaN        NaN      105.0
     Herkimer          NaN        NaN     4211.0
     Jefferson         NaN        NaN    11464.0
     Kings             NaN        NaN   284568.0
     Lewis             NaN        NaN     1997.0
     Livingston        NaN        NaN     1556.0
     Madison           NaN        NaN     4800.0
     Monroe            NaN        NaN    35077.0
     Montgomery        NaN        NaN     6078.0
     Nassau            NaN        NaN    42849.0
...                    ...        ...        ...
2014 Onondaga       6406.0    40219.0    46625.0
     Ontario        1281.0     6385.0     7666.0
     Orange         2978.0    21413.0    24391.0
     Orleans         373.0     2651.0     3024.0
     Oswego         1476.0    10162.0    11638.0
     Otsego          716.0     3474.0     4190.0
     Putnam          371.0     2691.0     3062.0
     Queens        19418.0   235147.0   254565.0
     Rensselaer     3416.0    17908.0    21324.0
     Richmond       5049.0    33776.0    38825.0
     Rockland       2392.0    14669.0    17061.0
     Saratoga       1863.0     7997.0     9860.0
     Schenectady    2564.0    20436.0    23000.0
     Schoharie       292.0     2316.0     2608.0
     Schuyler        486.0     1989.0     2475.0
     Seneca          324.0      963.0     1287.0
     St Lawrence    1801.0    16165.0    17966.0
     Statewide    219427.0  1845591.0  2065018.0
     Steuben        1764.0     9962.0    11726.0
     Suffolk       10370.0    84081.0    94451.0
     Sullivan       1080.0     7752.0     8832.0
     Tioga           175.0     2676.0     2851.0
     Tompkins        664.0     3267.0     3931.0
     Ulster         3481.0    15442.0    18923.0
     Warren          804.0     4038.0     4842.0
     Washington      448.0     3424.0     3872.0
     Wayne          1025.0     4504.0     5529.0
     Westchester    8009.0    62073.0    70082.0
     Wyoming         226.0     2033.0     2259.0
     Yates           289.0     1133.0     1422.0

[252 rows x 3 columns]

In [24]:
# If not allowed to use pivot table function, here is how to pivot step by step
ppv['Dual'] = ppv[ppv.dual_status == 'Dual'].medicaid_ppv_events
ppv['Non Dual'] = ppv[ppv.dual_status == 'Non Dual'].medicaid_ppv_events
ppv['Total'] = ppv[ppv.dual_status == 'Total '].medicaid_ppv_events
ppv.fillna(0,inplace=True)

In [18]:
values = ppv.dual_status.unique()
values

array(['Non Dual', u'Total ', u'Dual'], dtype=object)

In [19]:
# same as not in () .. statement in sql
ppv1[~ppv1.dual_status.isin(values)]

,year,county,dual_status,medicaid_ppv_events
0,2014,Albany,Non-Dual,28303
4,2014,Allegany,Non-Dual,3852
6,2014,Bronx,Non-Dual,310734
9,2014,Broome,Non-Dual,14854
14,2014,Cattaraugus,Non-Dual,7728
16,2014,Cayuga,Non-Dual,6794
19,2014,Chautauqua,Non-Dual,17505
22,2014,Chemung,Non-Dual,13452
25,2014,Chenango,Non-Dual,6627
29,2014,Clinton,Non-Dual,8476


In [29]:
#ppv.drop(['dual_status','medicaid_ppv_events'],axis = 1,inplace=True)
# aggregation function
ppv = ppv.groupby(['year','county']).sum()[['Dual','Non Dual', 'Total']].reset_index()
ppv.head()

,year,county,Dual,Non Dual,Total
0,2011,Albany,0.0,0.0,29612.0
1,2011,Allegany,0.0,0.0,3948.0
2,2011,Bronx,0.0,0.0,303162.0
3,2011,Broome,0.0,0.0,9121.0
4,2011,Cattaraugus,0.0,0.0,8012.0


In [46]:
# api to read another dataset
client = Socrata("data.ny.gov", None)
# (23735, 17) record
results = client.get("5bb2-yb85",limit = 30000)
income_tax = pd.DataFrame.from_records(results)
income_tax = income_tax[['tax_year','state','county','ny_agi_of_all_returns_in_thousands',
                        'taxable_income_of_all_returns_in_thousands']]
income_tax.columns = ['year','state','county','agi','taxable_income']
income_tax.head()

,year,state,county,agi,taxable_income
0,2014,New York,New York City - Bronx,28535.362,3335.61
1,2014,New York,New York City - Bronx,497520.476,40325.721
2,2014,New York,New York City - Bronx,1848243.085,537630.966
3,2014,New York,New York City - Bronx,2028682.259,1087516.097
4,2014,New York,New York City - Bronx,2259143.529,1486635.154


In [31]:
import warnings
warnings.filterwarnings('ignore')
# filter data, and change some values for county column
income_tax = income_tax[(income_tax.state == 'New York')& (income_tax.year > 2010)]
income_tax.loc[income_tax["county"] == 'New York City - Bronx','county'] = 'Bronx'
income_tax.loc[income_tax["county"] == 'New York City - Kings','county'] = 'Kings'
income_tax.loc[income_tax["county"] == 'New York City - Manhattan','county'] = 'Manhattan'
income_tax.loc[income_tax["county"] == 'New York City - Queens','county'] = 'Queens'
income_tax.loc[income_tax["county"] == 'New York City - Richmond','county'] = 'Richmond'
cols = ['agi', 'taxable_income']
income_tax[cols] = income_tax[cols].apply(pd.to_numeric, errors='coerce', axis=1)
# add a new column
income_tax['deductions'] = income_tax['agi'] - income_tax['taxable_income']
income_tax.head()

,year,state,county,agi,taxable_income,deductions
0,2014,New York,Bronx,28535.362,3335.610,25199.752
1,2014,New York,Bronx,497520.476,40325.721,457194.755
2,2014,New York,Bronx,1848243.085,537630.966,1310612.119
3,2014,New York,Bronx,2028682.259,1087516.097,941166.162
4,2014,New York,Bronx,2259143.529,1486635.154,772508.375


In [32]:
# get the mean 
income_tax = income_tax.groupby(['county','year']).mean().reset_index()
income_tax.head()

,county,year,agi,taxable_income,deductions
0,Albany,1999,9.316067e+05,714181.833333,217424.833333
1,Albany,2000,1.009254e+06,783491.250000,225762.666667
2,Albany,2001,9.758270e+05,750473.666667,225353.333333
3,Albany,2002,9.862647e+05,755584.166667,230680.500000
4,Albany,2003,1.026337e+06,792451.333333,233885.250000


In [33]:
# self join, just to see the output if you add suffixes
test = ppv.merge(ppv, left_on = ['county','year'], right_on = ['county','year'], how = 'inner', 
              suffixes = ('_t1','_t2'))

In [34]:
test.head()

,year,county,Dual_t1,Non Dual_t1,Total_t1,Dual_t2,Non Dual_t2,Total_t2
0,2011,Albany,0.0,0.0,29612.0,0.0,0.0,29612.0
1,2011,Allegany,0.0,0.0,3948.0,0.0,0.0,3948.0
2,2011,Bronx,0.0,0.0,303162.0,0.0,0.0,303162.0
3,2011,Broome,0.0,0.0,9121.0,0.0,0.0,9121.0
4,2011,Cattaraugus,0.0,0.0,8012.0,0.0,0.0,8012.0


In [35]:
# join the above data frame
df = ppv.merge(income_tax, left_on = ['county','year'], right_on = ['county','year'], how = 'inner', 
              suffixes = ('_t1','_t2'))

In [36]:
df.head()

,year,county,Dual,Non Dual,Total,agi,taxable_income,deductions
0,2011,Albany,0.0,0.0,29612.0,1.096756e+06,8.843990e+05,212357.428571
1,2011,Allegany,0.0,0.0,3948.0,9.482986e+04,6.831164e+04,26518.214286
2,2011,Bronx,0.0,0.0,303162.0,2.518053e+06,1.735262e+06,782790.357143
3,2011,Broome,0.0,0.0,9121.0,5.179209e+05,3.935487e+05,124372.142857
4,2011,Cattaraugus,0.0,0.0,8012.0,1.698241e+05,1.234909e+05,46333.142857


In [39]:
# mean for agi column, and sum for deductions column
income_tax.groupby(['county','year']).agg({'agi':np.mean,'deductions':sum})[['agi','deductions']].reset_index()

,county,year,agi,deductions
0,Albany,1999,9.316067e+05,217424.833333
1,Albany,2000,1.009254e+06,225762.666667
2,Albany,2001,9.758270e+05,225353.333333
3,Albany,2002,9.862647e+05,230680.500000
4,Albany,2003,1.026337e+06,233885.250000
5,Albany,2004,1.099927e+06,241077.333333
6,Albany,2005,1.125844e+06,250376.416667
7,Albany,2006,1.225914e+06,264476.916667
8,Albany,2007,1.101516e+06,234094.214286
9,Albany,2008,1.041000e+06,219394.642857


In [40]:
# same as window function lead()
df['next_year_deductions'] = df.sort_values(['county', 'year']).groupby('county')['deductions'].shift(-1)
df.sort_values(['county', 'year'])

,year,county,Dual,Non Dual,Total,agi,taxable_income,deductions,next_year_deductions
0,2011,Albany,0.0,0.0,29612.0,1.096756e+06,8.843990e+05,212357.428571,213977.681429
60,2012,Albany,0.0,0.0,31181.0,1.156916e+06,9.429381e+05,213977.681429,219653.871714
120,2013,Albany,5188.0,23733.0,28921.0,1.141294e+06,9.216400e+05,219653.871714,224350.069429
180,2014,Albany,4994.0,28303.0,33297.0,1.204678e+06,9.803279e+05,224350.069429,NaN
1,2011,Allegany,0.0,0.0,3948.0,9.482986e+04,6.831164e+04,26518.214286,26220.548000
61,2012,Allegany,0.0,0.0,4115.0,9.882933e+04,7.260878e+04,26220.548000,26526.196143
121,2013,Allegany,911.0,3361.0,4272.0,9.978862e+04,7.326242e+04,26526.196143,26283.268714
181,2014,Allegany,864.0,3852.0,4716.0,1.078800e+05,8.159668e+04,26283.268714,NaN
2,2011,Bronx,0.0,0.0,303162.0,2.518053e+06,1.735262e+06,782790.357143,777193.260000
62,2012,Bronx,0.0,0.0,305888.0,2.558104e+06,1.780911e+06,777193.260000,805035.384429


In [41]:
# rank(method=’first’) same as row_number() in sql
# method=’min’ same as rank(), method = ‘dense’ same as dense_rank()
df['rank'] = df.groupby('county')['deductions'].rank(method = 'first', ascending = False)
df.sort_values(['county', 'year'])

,year,county,Dual,Non Dual,Total,agi,taxable_income,deductions,next_year_deductions,rank
0,2011,Albany,0.0,0.0,29612.0,1.096756e+06,8.843990e+05,212357.428571,213977.681429,4.0
60,2012,Albany,0.0,0.0,31181.0,1.156916e+06,9.429381e+05,213977.681429,219653.871714,3.0
120,2013,Albany,5188.0,23733.0,28921.0,1.141294e+06,9.216400e+05,219653.871714,224350.069429,2.0
180,2014,Albany,4994.0,28303.0,33297.0,1.204678e+06,9.803279e+05,224350.069429,NaN,1.0
1,2011,Allegany,0.0,0.0,3948.0,9.482986e+04,6.831164e+04,26518.214286,26220.548000,2.0
61,2012,Allegany,0.0,0.0,4115.0,9.882933e+04,7.260878e+04,26220.548000,26526.196143,4.0
121,2013,Allegany,911.0,3361.0,4272.0,9.978862e+04,7.326242e+04,26526.196143,26283.268714,1.0
181,2014,Allegany,864.0,3852.0,4716.0,1.078800e+05,8.159668e+04,26283.268714,NaN,3.0
2,2011,Bronx,0.0,0.0,303162.0,2.518053e+06,1.735262e+06,782790.357143,777193.260000,3.0
62,2012,Bronx,0.0,0.0,305888.0,2.558104e+06,1.780911e+06,777193.260000,805035.384429,4.0


In [42]:
# sum by country
df2 = df.groupby('county')['deductions'].sum().reset_index(name = 'sum')
df2.head()

,county,sum
0,Albany,8.703391e+05
1,Allegany,1.055482e+05
2,Bronx,3.197094e+06
3,Broome,5.090993e+05
4,Cattaraugus,1.851089e+05


In [44]:
# see ratio 
df3 = df.merge(df2, on = 'county', how = 'inner')
df3['ratio'] = df3['deductions']/df3['sum']
df3.head()

,year,county,Dual,Non Dual,Total,agi,taxable_income,deductions,next_year_deductions,rank,sum,ratio
0,2011,Albany,0.0,0.0,29612.0,1.096756e+06,884399.000000,212357.428571,213977.681429,4.0,870339.051143,0.243994
1,2012,Albany,0.0,0.0,31181.0,1.156916e+06,942938.136571,213977.681429,219653.871714,3.0,870339.051143,0.245856
2,2013,Albany,5188.0,23733.0,28921.0,1.141294e+06,921640.012000,219653.871714,224350.069429,2.0,870339.051143,0.252377
3,2014,Albany,4994.0,28303.0,33297.0,1.204678e+06,980327.876714,224350.069429,NaN,1.0,870339.051143,0.257773
4,2011,Allegany,0.0,0.0,3948.0,9.482986e+04,68311.642857,26518.214286,26220.548000,2.0,105548.227143,0.251243
